In [5]:
!pip install evaluate

# GPT2-XL Evaluation

In [2]:
from transformers import GPT2Tokenizer, AutoModelForCausalLM
import numpy as np
from evaluate import load

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
model = AutoModelForCausalLM.from_pretrained("gpt2-xl").cuda()
tokenizer.pad_token_id = tokenizer.eos_token_id

perplexity = load("perplexity", module_type="metric")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
outputs = []
input_ids = tokenizer(["Once upon a time,"], return_tensors="pt").to("cuda:0")
for _ in range(8):
    outputs += model.generate(**input_ids, num_return_sequences=4, max_length=256, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [12]:
print("\n\n".join(tokenizer.batch_decode(outputs, skip_special_tokens=True)[:10]))

Once upon a time, in a time far, far, away…

This is the beginning of something new and unique, something I hope to show you folks one evening of the year: the time of season 3 will officially (and by the way, before you ask, we don't have a set date for it yet) be titled "The Next Step." What that actually means, I have no idea. What I do know is that the creative team and I believe it's a great time to bring a new entry to the long-running show, and it will take place on the same night as season 2 (but will take place in a much different time period). We were talking about this a bit recently and we both feel it's a very different time period than the first three seasons with a lot of other variables such as the fact that the show is the same age and there just aren't as many of the main characters to worry about. We were also talking a little bit about a few of our favourite moments in the first three seasons because they definitely take place out in the time periods we're working i

In [13]:
predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = perplexity.compute(predictions=predictions, batch_size=16, model_id='gpt2-large', device="cuda")
print('mean_perplexity:', results['mean_perplexity'])

  0%|          | 0/2 [00:00<?, ?it/s]

mean_perplexity: 10.452028013765812


# My Boutique LM Evaluation

In [2]:
!git clone https://github.com/yaraksen/murky_lm.git
%cd murky_lm

Cloning into 'murky_lm'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 61 (delta 34), reused 45 (delta 18), pack-reused 0
Receiving objects: 100% (61/61), 206.40 KiB | 1.38 MiB/s, done.
Resolving deltas: 100% (34/34), done.
/kaggle/working/murky_lm


In [8]:
!git pull origin

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), 275 bytes | 275.00 KiB/s, done.
From https://github.com/yaraksen/murky_lm
   ac4e142..0925942  master     -> origin/master
Updating ac4e142..0925942
Fast-forward
 test.py | 2 --
 1 file changed, 2 deletions(-)


In [9]:
%load_ext autoreload
%autoreload 2

from murky_lm.test import murkylm_ppl
from transformers import GPT2Tokenizer, AutoModelForCausalLM
import numpy as np
from evaluate import load

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
murkylm_ppl("/kaggle/input/epoch12/checkpoint-epoch12.pth", N=32, temp=1.0)

cuda:0


Generating...:   0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:544: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._transformer_encoder_layer_fwd(
Generating...: 100%|██████████| 1/1 [00:11<00:00, 11.49s/it]


  0%|          | 0/4 [00:00<?, ?it/s]

[160, 189, 241, 145, 183, 166, 226, 158, 220, 197, 187, 159, 158, 205, 138, 183, 219, 147, 216, 255, 212, 170, 159, 195, 167, 230, 202, 185, 137, 140, 211, 190]
Once upon a time, there was a little girl named Lily. She loved to play outside in the snow. One day, she saw her friend Timmy walking by. "Hi Timmy!" Lily said. "Hi Lily!" Timmy replied. Lily noticed that Timmy was wearing a white hat. "Why are you wearing that hat?" she asked. "I'm wearing it to keep my head warm," Timmy said. Lily thought that was cool. She started to lick her lips because it was so warm. "Your lips are so cold yuck," she complained. Timmy laughed. "It's not woofing, silly!" he said. Lily and Timmy played together in the snow, making snow angels and having snowball fights. They had so much fun together.

One day in the evening there was a little girl called Daisy. Daisy was three and her name was Lucy. Daisy was in the park playing with her friends when suddenly a loud rumble filled the air. It was thunder! 

In [11]:
murkylm_ppl("/kaggle/input/epoch12/checkpoint-epoch12.pth", N=32, temp=0.9)

cuda:0


Generating...: 100%|██████████| 1/1 [00:11<00:00, 11.46s/it]


  0%|          | 0/4 [00:00<?, ?it/s]

[197, 128, 201, 200, 144, 232, 121, 184, 170, 173, 154, 197, 175, 152, 147, 238, 193, 204, 191, 226, 153, 168, 207, 178, 245, 174, 255, 228, 230, 169, 233, 169]
It was a cold day, and the day was covered in snow. A three year old child was outside, playing in the snow, building a snowman. But as soon as it got there, a gust of wind blew and scattered the snow all over the ground. The child looked up and saw the snow everywhere! The child smiled and ran to the top of the hill to start building. The child began putting their hands together, putting their feet together and trying to make snowflakes. Suddenly, an old man appeared. He took an icy look around the boy and said, "That snow is dangerous. You must come down!". The child screamed and ran down the hill as fast as they could. They ran until they were too far away to catch any fires. The old man laughed and said, "Good time! That man is a dangerous friend". The child smiled and ran back down the hill, excited to start building a sno

In [12]:
murkylm_ppl("/kaggle/input/epoch12/checkpoint-epoch12.pth", N=32, temp=0.8)

cuda:0


Generating...: 100%|██████████| 1/1 [00:11<00:00, 11.48s/it]


  0%|          | 0/4 [00:00<?, ?it/s]

[201, 152, 132, 119, 222, 255, 195, 192, 131, 190, 114, 243, 182, 198, 177, 242, 255, 153, 202, 222, 142, 152, 118, 214, 159, 255, 228, 204, 225, 159, 187, 255]
Once upon a time, there was a little girl named Lily. She loved to play with her toys on the couch. One day, Lily's mom brought home a new couch. It was big and soft, and it was perfect for Lily's room. But Lily didn't understand how to put it in her room. She asked her mom, "How do I put my new couch in my room?" Her mom replied, "You can put it in your room, but you have to be serious and do it slowly. It will take some time, but some time." Lily tried to be serious, but she didn't understand. She played with her toys and read her books on the couch. Soon, it was time for bed, and Lily's mom came to check on her. She saw how serious Lily was and said, "I understand how to put your couch in your room." Lily was happy and slept on her new couch with a big smile on her face.

Once upon a time, there was a funny dog named Spot. S

In [13]:
murkylm_ppl("/kaggle/input/epoch12/checkpoint-epoch12.pth", N=32, temp=0.7)

cuda:0


Generating...: 100%|██████████| 1/1 [00:11<00:00, 11.49s/it]


  0%|          | 0/4 [00:00<?, ?it/s]

[135, 163, 255, 185, 199, 209, 154, 220, 255, 182, 156, 153, 189, 194, 154, 158, 193, 167, 180, 175, 165, 101, 198, 217, 170, 221, 137, 171, 168, 165, 222, 151]
Once upon a time, there was a little boy named Tim. Tim loved to play with his toy cars. One day, he found a big cardboard box. It was very tough to move around in. Tim wanted to show his mom the big cardboard box. He tried to lift it, but it was too heavy. So, he asked his dad to help him. Together, they lifted the box and opened it. Inside, Tim found a big shovel. He was so happy! He used the shovel to make a road for his cars on the road. Now, Tim could play with his cars on the road. He drove them all around and had lots of fun.

Once upon a time, there was a little monster. The monster loved to play with his friends. They liked to run and jump all day long. The monster was very happy. One day, the monster saw a big, round thing. He did not know what it was. The monster thought it was a ball. He tried to play with it. But i